# Implementing Basic RAG QA with Langchain
This notebook demonstrates how to implement a basic Retrieval-Augmented Generation (RAG) chain using our FAQ data. The overall approach is as follows:
1. Load FAQ Data & Structure Metadata
2. Create Embeddings for the FAQ Data using Amazon's Titan Embeddings model, and save these embeddings locally using Chroma.
3. Create a Question Answering (QA) chain which retrieves context based on the embeddings saved in Chroma, serving these as context to the Amazon Titan Express LLM to answer the provided user prompt.
4. Format the response so that source materials can be cited.

This implementation mostly follows these Langchain tutorials:
- https://python.langchain.com/docs/modules/data_connection/document_loaders/json#using-jsonloader

In [1]:
# Define metadata extraction function so we can filter on sections and return deep links as sources
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["section"] = record.get("section")
    metadata["source"] = record.get("deep_link")
    
    return metadata

In [2]:
# Import JSON FAQ File using JSONLoader
from langchain_community.document_loaders import JSONLoader
from pprint import pprint

file_path='../data/processed/faq_data/FA_NSYR.json'

loader = JSONLoader(
    file_path=file_path,
    jq_schema=".[]",
    content_key="answer",
    metadata_func=metadata_func
)

data = loader.load()

In [3]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)
persistDirectory = './data/processed/faq_data/vectordata/FA_NSYR'
vectorstore = Chroma.from_documents(
    documents=data, embedding=embeddings, persist_directory=persistDirectory)
vectorstore.persist()

Run the cells below to re-use already-generated vector embeddings.

In [4]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)

persistDirectory = './data/processed/faq_data/vectordata/FA_NSYR'
# Retrieve and generate answers using relevant FAQs
vectordb = Chroma(persist_directory=persistDirectory,
                  embedding_function=embeddings)

retriever = vectordb.as_retriever()

In [5]:
from utils import CreateInferenceModifier # Import the function from utils.py
from langchain import hub
from langchain_community.llms import Bedrock
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt_template = """Drawing from your knowledge base, answer the question below.
If you don't know the answer from the provided context, tell me that your training materials don't include this information.
Keep the answer as concise and relevant to the question as possible.

Knowledge Base: {context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(prompt_template)


# Define the universal set of modifier parameters
modifiers = {"max_tokens": 4000,
             "temperature": 0.2,
             "top_k": 250,
             "top_p": 1,
             "stop_sequences": ["\n\nHuman"],
             }


llm = Bedrock(model_id="anthropic.claude-v2:1",
              model_kwargs=CreateInferenceModifier("claude", params=modifiers))

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [7]:
# Define a function to extract unique URLs used in the retrieved source materials.
from IPython.display import Markdown, display
def extract_unique_urls(response):
    unique_urls = set()  # Use a set to store unique URLs
    
    # Iterate through each document in the 'context'
    for document in response['context']:
        source_url = document.metadata['source']  # Extract the 'source' URL
        unique_urls.add(source_url)  # Add the URL to the set
    
    # Convert the set of unique URLs to a string
    urls_string = '; '.join(unique_urls)
    
    return urls_string


# Invoke the chain and print the response and sources.
response = rag_chain_with_source.invoke(
    "چه مراحلی باید برای درخواست پناهندگی در ترکیه طی کنم؟")
answer = response["answer"]
prettyResponse = f"## Answer: \n\n {answer} \n\n ## Sources: \n\n {extract_unique_urls(response)}"
Markdown(prettyResponse)

## Answer: 

  براساس اطلاعات ارائه شده در متن، برای درخواست پناهندگی در ترکیه باید مراحل زیر را طی کنید:

1. درخواست پناهندگی خود را به عنوان "درخواست حمایت بین‌المللی" یا برای شهروندان سوریه به عنوان "درخواست حمایت موقت" به اداره مهاجرت استان محل اقامت خود ارائه دهید.

2. دلایل ناتوانی بازگشت به کشور مبدأ را با جزئیات توضیح دهید. هرگونه مدرک شناسایی یا مدارکی که نشان دهنده ترس از آزار و اذیت باشد را ارائه کنید.

3. منتظر بمانید تا درخواست شما توسط مقامات ترکیه بررسی و تصمیم‌گیری شود.

4. در صورت نیاز به راهنمایی، با شماره تماس مرکز حقوق پناهندگان ترکیه تماس بگیرید.

این مراحل اصلی‌ترین گام‌ها برای درخواست پناهندگی در ترکیه است. امیدوارم پاسخ مفید بوده باشد. 

 ## Sources: 

 https://multecihaklari.info/fa/services/%d8%ad%d9%82%d9%88%d9%82-%d9%88-%d8%b1%d9%88%d9%86%d8%af-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%af%d8%b1-%d8%a8%d8%a7%d8%b2%d8%af%d8%a7%d8%b4%d8%aa-%d8%a7%d8%af%d8%a7%d8%b1%db%8c/?section=questions&question=7; https://multecihaklari.info/fa/services/%d8%ab%d8%a8%d8%aa-%d9%86%d8%a7%d9%85-%d9%88%d8%b6%d8%b9%db%8c%d8%aa-2/?section=questions&question=18; https://multecihaklari.info/fa/services/%d8%ac%d8%b2%d9%88%d9%87-%d8%a7%d8%b7%d9%84%d8%a7%d8%b9%d8%a7%d8%aa%db%8c-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%ae%d8%b1%d8%af%d8%b3%d8%a7%d9%84-%d8%a8%d8%af%d9%88%d9%86-%d9%87/?section=questions&question=13; https://multecihaklari.info/fa/services/%d8%ab%d8%a8%d8%aa-%d9%86%d8%a7%d9%85-%d9%88%d8%b6%d8%b9%db%8c%d8%aa-2/?section=questions&question=48

In [8]:
def query_asylum_system(question):
    from IPython.display import Markdown, display
    # Invoke the LLM chain with the provided question
    response = rag_chain_with_source.invoke(question)

    # Extract the answer from the response
    answer = response["answer"]

    # Format the response using Markdown
    pretty_response = f"## Answer: \n\n{answer}\n\n## Sources:\n\n{extract_unique_urls(response)}"
    return pretty_response

In [9]:
Markdown(query_asylum_system(
    "آیا می‌توانید درباره حقوقی که به عنوان یک پناهجو در ترکیه دارم توضیح دهید؟"))

## Answer: 

 بله، می‌توانم در مورد حقوقی که به عنوان یک پناهجو در ترکیه دارید، توضیحاتی ارائه دهم:

درخواست حمایت بین‌المللی به شما حق ماندن قانونی در ترکیه را می‌دهد. شما از خطر اخراج به کشورتان یا هر کشور دیگری که در معرض خطر آن هستید، در امان خواهید بود. همچنین می‌توانید از خدماتی مانند مراقبت‌های بهداشتی و آموزشی به صورت رایگان بهره‌مند شوید.

با درخواست پناهندگی، تا زمانی که درخواست شما در حال بررسی است، به زور به کشور مبدأ بازگردانده نمی‌شوید. همچنین با یک مدرک شناسایی، می‌توانید به طور قانونی در ترکیه بمانید.

سوری‌ها می‌توانند از حقوقی مانند آموزش و مراقبت‌های بهداشتی رایگان برخوردار شوند. درخواست‌کنندگان حمایت بین‌المللی نیز ممکن است از بیمه درمانی رایگان به مدت یک سال بهره‌مند شوند.

خردسالان بدون همراه نیز صرف نظر از وضعیت حمایتی‌شان، می‌توانند از حقوق و خدمات متعددی بهره‌مند شوند.

امیدوارم این توضیحات به شما در درک حقوقتان به عنوان یک پناهجو در ترکیه کمک کند. اگر سؤال دیگری دارید، خوشحال می‌شوم پاسخ دهم.

## Sources:

https://multecihaklari.info/fa/services/%da%a9%d9%85%da%a9-%d9%82%d8%a7%d9%86%d9%88%d9%86%db%8c-2/?section=questions&question=1; https://multecihaklari.info/fa/services/%d8%ab%d8%a8%d8%aa-%d9%86%d8%a7%d9%85-%d9%88%d8%b6%d8%b9%db%8c%d8%aa-2/?section=questions&question=3; https://multecihaklari.info/fa/services/%d8%ac%d8%b2%d9%88%d9%87-%d8%a7%d8%b7%d9%84%d8%a7%d8%b9%d8%a7%d8%aa%db%8c-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%ae%d8%b1%d8%af%d8%b3%d8%a7%d9%84-%d8%a8%d8%af%d9%88%d9%86-%d9%87/?section=questions&question=18; https://multecihaklari.info/fa/services/%d8%ac%d8%b2%d9%88%d9%87-%d8%a7%d8%b7%d9%84%d8%a7%d8%b9%d8%a7%d8%aa%db%8c-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%ae%d8%b1%d8%af%d8%b3%d8%a7%d9%84-%d8%a8%d8%af%d9%88%d9%86-%d9%87/?section=questions&question=14

In [10]:
Markdown(query_asylum_system(
    "فرآیند پناهندگی معمولاً چقدر زمان می‌برد در ترکیه؟"))

## Answer: 

 متأسفانه در مورد مدت زمان لازم برای فرآیند پناهندگی در ترکیه اطلاعات دقیقی در متن فوق وجود ندارد. 

در متن توضیح داده شده است که پس از ثبت‌نام برای حمایت موقت یا بین‌المللی، شما حق دارید تا زمانی که درخواست شما در حال بررسی است و تا زمانی که در نهایت رد شود، به زور به کشور مبدأ خود بازگردانده نشوید. 

اما در مورد مدت زمان لازم برای بررسی درخواست پناهندگی اطلاعات دقیقی ارائه نشده است. متأسفانه در این متن نمی‌توانم به سؤال شما در مورد مدت زمان فرآیند پناهندگی در ترکیه پاسخ دهم.

## Sources:

https://multecihaklari.info/fa/services/%d8%ad%d9%82%d9%88%d9%82-%d9%88-%d8%b1%d9%88%d9%86%d8%af-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%af%d8%b1-%d8%a8%d8%a7%d8%b2%d8%af%d8%a7%d8%b4%d8%aa-%d8%a7%d8%af%d8%a7%d8%b1%db%8c/?section=questions&question=31; https://multecihaklari.info/fa/services/%d8%ac%d8%b2%d9%88%d9%87-%d8%a7%d8%b7%d9%84%d8%a7%d8%b9%d8%a7%d8%aa%db%8c-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%ae%d8%b1%d8%af%d8%b3%d8%a7%d9%84-%d8%a8%d8%af%d9%88%d9%86-%d9%87/?section=questions&question=17; https://multecihaklari.info/fa/services/%d8%ac%d8%b2%d9%88%d9%87-%d8%a7%d8%b7%d9%84%d8%a7%d8%b9%d8%a7%d8%aa%db%8c-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%ae%d8%b1%d8%af%d8%b3%d8%a7%d9%84-%d8%a8%d8%af%d9%88%d9%86-%d9%87/?section=questions&question=14; https://multecihaklari.info/fa/services/%da%a9%d8%a7%d8%b1-2/?section=questions&question=5

In [11]:
Markdown(query_asylum_system(
    "پناهجویان چه نوع حمایتی می‌توانند در ترکیه دریافت کنند؟"))

## Answer: 

 بر اساس اطلاعات ارائه شده در متن، پناهجویان می‌توانند انواع زیر از حمایت‌ها را در ترکیه دریافت کنند:

- اسکان رایگان برای کودکان جداشده و بدون همراه در تسهیلات دولتی 
- امکان اسکان رایگان برای تعداد محدودی از درخواست‌کنندگان پناهندگی که به‌طور ویژه آسیب‌پذیر تلقی می‌شوند
- دسترسی به خدمات پشتیبانی و کمک‌رسانی توسط برخی سازمان‌های مردم‌نهاد
- برخورداری از حقوق و خدمات اساسی مانند مراقبت‌های پزشکی و آموزش در طول روند رسیدگی به درخواست پناهندگی

بنابراین، انواع حمایت‌های ارائه شده به پناهجویان در ترکیه عبارتند از: اسکان رایگان برای گروه‌های خاص، خدمات پشتیبانی توسط NGOها، و برخورداری از حقوق و خدمات پایه در طول روند رسیدگی به پرونده پناهندگی.

## Sources:

https://multecihaklari.info/fa/services/%d8%ab%d8%a8%d8%aa-%d9%86%d8%a7%d9%85-%d9%88%d8%b6%d8%b9%db%8c%d8%aa-2/?section=questions&question=18; https://multecihaklari.info/fa/services/%d8%ab%d8%a8%d8%aa-%d9%86%d8%a7%d9%85-%d9%88%d8%b6%d8%b9%db%8c%d8%aa-2/?section=questions&question=47; https://multecihaklari.info/fa/services/%d8%ab%d8%a8%d8%aa-%d9%86%d8%a7%d9%85-%d9%88%d8%b6%d8%b9%db%8c%d8%aa-2/?section=questions&question=0; https://multecihaklari.info/fa/services/%d8%ac%d8%b2%d9%88%d9%87-%d8%a7%d8%b7%d9%84%d8%a7%d8%b9%d8%a7%d8%aa%db%8c-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%ae%d8%b1%d8%af%d8%b3%d8%a7%d9%84-%d8%a8%d8%af%d9%88%d9%86-%d9%87/?section=questions&question=16

In [15]:
Markdown(query_asylum_system(
    "What happens if my asylum application is denied in Turkey?"))

## Answer: 

 Based on the knowledge base provided, if your asylum application is denied in Turkey:

<p>
- The authorities are required to communicate the negative decision and reasons to you or your legal representative. 

- You have the right to appeal the negative decision. If you are in administrative detention, you typically need to appeal to the administrative court within 15 days of being notified of the decision.

- You are allowed to stay in Turkey until there is a final decision on any appeal you file within the time limits. However, you need to notify the Provincial Directorate of Migration Management about your appeal.

- You can access legal support to help with your appeal, such as by calling the Detention Hotline of Refugee Rights Turkey.

So in summary, you have the right to appeal a negative asylum decision in Turkey, and can stay in the country during that appeal process. There are also options to access legal help with filing and pursuing that appeal.

</p>

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=34; https://multecihaklari.info/services/rights-and-procedures-for-persons-in-administrative-detention/?section=questions&question=15; https://multecihaklari.info/services/registration-status-2/?section=questions&question=49

In [17]:
Markdown(query_asylum_system(
    "Can asylum seekers work in Turkey while their application is pending?"))

## Answer: 

 <p>Yes, according to the information provided, asylum seekers can apply for a work permit in Turkey while their application is pending, with some conditions:</p>

<ul>
<li>Asylum seekers can apply for a work permit 6 months after submitting their international protection application.</li>
<li>Work permit requests for asylum seekers are usually submitted by their employer via the e-Government Gateway system.</li>
<li>If the asylum seeker has established their own business in Turkey, they can apply for an "independent work permit" themselves.</li>
</ul>

<p>So in summary, yes asylum seekers can legally work in Turkey while their application is pending, if they meet the requirements around timing and submitting the proper work permit application.</p>

The knowledge base covers the conditions and process for asylum seekers to obtain work permits in Turkey, so I've summarized the key points related to your question in my answer. Let me know if you need any clarification or have additional questions!

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=3; https://multecihaklari.info/services/labor-market-2/?section=questions&question=5; https://multecihaklari.info/services/labor-market-2/?section=questions&question=0; https://multecihaklari.info/services/registration-status-2/?section=questions&question=49

In [18]:
Markdown(query_asylum_system(
    "Where can I get information about the status of my asylum application in Turkey?"))

## Answer: 

 Unfortunately, the knowledge base provided does not include clear information to directly answer the question about where to get information on the status of an asylum application in Turkey. 

The knowledge base talks about what information authorities should provide when an administrative detention order is revoked, including information on one's status in Turkey, rights and entitlements. It also mentions contacting Refugee Rights Turkey or other NGOs to get information about obligations and procedures to follow after release from detention.

However, there is no specific information provided on where to check on the status of an asylum application that has already been submitted. The knowledge base does not seem to cover that particular topic. So I do not have enough context in the provided materials to directly answer the question asked. I apologize that I cannot be more helpful here based on what was provided.

## Sources:

https://multecihaklari.info/services/rights-and-procedures-for-persons-in-administrative-detention/?section=questions&question=21; https://multecihaklari.info/services/registration-status-2/?section=questions&question=41; https://multecihaklari.info/services/rights-and-procedures-for-persons-in-administrative-detention-2/?section=questions&question=21

In [19]:
Markdown(query_asylum_system(
    "Are there specific documents I need to provide when applying for asylum in Turkey?"))

## Answer: 

 Based on the information provided, if you are a Syrian national applying for temporary protection in Turkey, it is helpful to present your ID documents from Syria when applying. 

If you are applying for international protection from a country other than Syria, while not required, it will be helpful to share with the migration authorities any ID documents you have brought with you. You are also expected to explain in detail the reasons why you cannot go back to your country of origin.

The knowledge base does not specify any other required documents for applying for asylum in Turkey. It focuses more on explaining your situation and why you fear returning to your home country.

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=3; https://multecihaklari.info/services/rights-and-procedures-for-unaccompanied-minors/?section=questions&question=17; https://multecihaklari.info/services/registration-status-2/?section=questions&question=49